CS524: Introduction to Optimization Project Final Report
======================================

## Zachary Collins<br> University of Wisconsin-Madison 

## December 21, 2023
--------------
<br>

# Introduction to the Airline Service Ratings

<br>

## What is the issue being addressed?

There are many airlines that can be taken and many people have opinions on which has the best service. I aim to find out which airline has the best amenities, service, and whether flyers have biases about companies that influence their ratings. I based this problem off the support vectors problem in question 1 of homework 9, but made large changes in the data being evaluating through regression. I wanted to make sure I am using real life data to compare all the airlines and amenities.


## Where does the data come fram and how is it obtained?

I will be using data from Skytrax that github user Juhibojani compiled using data scraping of Skytrax’s reviews. These reviews are moderated by the website to make sure they only contain only relevant information and aren’t harassing the airline with the review.

found here: https://github.com/Juhibhojani/Airline-Reviews-

## What is the optimization problem underlying this project?

The optimization of this problem will be based similar to the support vectors problem in question 1 of homework 9 which is a classification problem. I plan on performing this problem multiple times by using three different datasets that will include different factors that influence the 'overall_rating' and 'recommended'. I will perform the first only using the amenities on the ship as influence for 'overall_rating' and 'recommended'. I will then perform the second using only company name as the influence for 'overall_rating' and 'recommended'. Then I will perform the last using both the company name and ship amenities.


## What are the deliverables?

- Import the data into GAMS
- Format the data for ease of use in GAMS
- Perform the first optimization, record the results/error
- Perform the second optimization, record the results/error
- Perform the final optimization, record the results/error
- Display the amenities from least to most influential visually.
- Display the airlines from least to most influential visually.
- Display the airlines from least to most influential visually.
- Final comparison / conclusion on results

## Other points

The data that is scraped from the Github is consistently updated.